# import libraries

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy 
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,f1_score

# training and testing functions

In [2]:
import time

def train_model(train_loader, learning_rate, num_epochs):
    # Print training details
    batch_size = train_loader.batch_size
    print(f"Batch Size: {batch_size}")
    print(f"Number of Epochs: {num_epochs}")
    print(f"Learning Rate (starting): {learning_rate}")

    # Modify the model's final layer
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward propagation
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward propagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print loss per epoch
        print(f"Epoch: [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")
    # Calculate latency for a single sample
    model.eval() 
    with torch.no_grad():
        start_time = time.time()
        for images, _ in train_loader:
            images = images.to(device)
            # Simulate processing a single sample
            for i in range(images.size(0)):
                _ = model(images[i].unsqueeze(0))
            break  # Measure for one batch
        end_time = time.time()

    # Latency per sample
    latency_per_sample = (end_time - start_time) / images.size(0)
    print(f"Latency per sample: {latency_per_sample * 1000:.2f} ms")
    

def test_model(test_loader,model):
    
    model.eval()
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for images,labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs.data,1)
            total+=labels.size(0)
            correct += (predicted == labels).sum().item()
            
            predicted_labels.extend(predicted.to('cpu'))
            true_labels.extend(labels.to('cpu'))

    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return (predicted_labels,true_labels)              

# setting the device for execution - gpu or cpu

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyperparameters

In [4]:
num_classes = 10
num_epochs = 10
batch_size = 32
learning_rate =0.0001

# pre processing functions

In [5]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

## downloading public dataset - CIFAR10

In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transforms, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=test_transforms, download=True)

# loading dataset by batch size to be used by pytorch for training and testing

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)


# loading ResNet18 model

In [ ]:
model = models.resnet18(pretrained=True)

# training the model

In [ ]:
train_model(train_loader,learning_rate,num_epochs)

# testing the model


In [ ]:
predictedLabels,trueLabels = test_model(test_loader,model)

# plotting confusion matrix

In [ ]:
cm = confusion_matrix(trueLabels, predictedLabels)
fig, ax = plt.subplots(figsize=(8,8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=range(10))
disp.plot(ax=ax, cmap='Blues', colorbar=True)
plt.title("Confusion Matrix for 10 Classes")
plt.show()


# calculating f1 score

In [ ]:
f1_score_data = f1_score(trueLabels, predictedLabels,average=None)
for i in f1_score_data:
    print(f'Class {numpy.where(f1_score_data==i)[0][0]} : {i}')

# update batch size

In [13]:
batch_size = 64 # previously 32

# update dataloader to set the updated batch size

In [14]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

# training the model

In [ ]:
train_model(train_loader,learning_rate,num_epochs)

# testing the model

In [ ]:
predictedLabels,trueLabels = test_model(test_loader,model)

In [ ]:
cm = confusion_matrix(trueLabels, predictedLabels)
fig, ax = plt.subplots(figsize=(8,8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=range(10))
disp.plot(ax=ax, cmap='Blues', colorbar=True)
plt.title("Confusion Matrix for 10 Classes")
plt.show()


In [ ]:
f1_score_data = f1_score(trueLabels, predictedLabels,average=None)
for i in f1_score_data:
    print(f'Class {numpy.where(f1_score_data==i)[0][0]} : {i}')

# updating num of epochs

In [19]:
num_epochs = 50

In [ ]:
train_model(train_loader,learning_rate,num_epochs)

In [ ]:
predictedLabels,trueLabels = test_model(test_loader,model)

In [ ]:
cm = confusion_matrix(trueLabels, predictedLabels)
fig, ax = plt.subplots(figsize=(8,8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=range(10))
disp.plot(ax=ax, cmap='Blues', colorbar=True)
plt.title("Confusion Matrix for 10 Classes")
plt.show()


In [ ]:
f1_score_data = f1_score(trueLabels, predictedLabels,average=None)
for i in f1_score_data:
    print(f'Class {numpy.where(f1_score_data==i)[0][0]} : {i}')

In [24]:
learning_rate = 0.001


In [ ]:
train_model(train_loader,learning_rate,num_epochs)

In [ ]:
predictedLabels,trueLabels = test_model(test_loader,model)

In [ ]:
cm = confusion_matrix(trueLabels, predictedLabels)
fig, ax = plt.subplots(figsize=(8,8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=range(10))
disp.plot(ax=ax, cmap='Blues', colorbar=True)
plt.title("Confusion Matrix for 10 Classes")
plt.show()


In [ ]:
f1_score_data = f1_score(trueLabels, predictedLabels,average=None)
for i in f1_score_data:
    print(f'Class {numpy.where(f1_score_data==i)[0][0]} : {i}')

# performing image augmentations

In [29]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4),   # Random crop with padding
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    transforms.RandomRotation(15),         # Random rotation within 15 degrees
    transforms.ToTensor(),                 # Convert to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
])

# Test data transformations (no augmentation)
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
])


In [30]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
train_model(train_loader,learning_rate,num_epochs)

In [ ]:
predictedLabels,trueLabels = test_model(test_loader,model)

In [ ]:
cm = confusion_matrix(trueLabels, predictedLabels)
fig, ax = plt.subplots(figsize=(8,8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=range(10))
disp.plot(ax=ax, cmap='Blues', colorbar=True)
plt.title("Confusion Matrix for 10 Classes")
plt.show()


In [ ]:
f1_score_data = f1_score(trueLabels, predictedLabels,average=None)
for i in f1_score_data:
    print(f'Class {numpy.where(f1_score_data==i)[0][0]} : {i}')